In [103]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

diabetes = pd.read_csv('data/diabetes_data.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


Pregnancies — количество беременностей.

Glucose — концентрация глюкозы в плазме через два часа при пероральном тесте на толерантность к глюкозе.

BloodPressure — диастолическое артериальное давление (мм рт. ст.).

SkinThickness — толщина кожной складки трицепса (мм).

Insulin — двухчасовой сывороточный инсулин (ме Ед/мл).

BMI — индекс массы тела (\(\frac{вес\ в\ кг}{(рост\ в\ м)^2}\)).

DiabetesPedigreeFunction — функция родословной диабета (чем она выше, тем выше шанс наследственной заболеваемости).

Age — возраст.

Outcome — наличие диабета (0 — нет, 1 — да).

In [104]:
diabetes_dedupped = diabetes.drop_duplicates()
diabetes_dedupped.shape

(768, 10)

In [105]:
low_info_cols = []
for col in diabetes_dedupped.columns:
    top_freq = diabetes_dedupped[col].value_counts(normalize=True).max()
    uniq_ratio = diabetes_dedupped[col].nunique() / diabetes_dedupped[col].count()
    if top_freq > 0.99:
        print(f'Признак {col} имеет {round(top_freq * 100, 2)}% одинаковых значений')
        low_info_cols.append(col)
    if uniq_ratio > 0.95:
        print(f'Признак {col} имеет {round(uniq_ratio * 100, 2)}% уникальных значений')
        low_info_cols.append(col)
    
    diab_clean = diabetes_dedupped.drop(columns = low_info_cols, axis = 1)


Признак Gender имеет 100.0% одинаковых значений


In [106]:
zero_col_list = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
for col in zero_col_list:
    diab_clean.loc[(diabetes_dedupped[col] == 0), col] = np.nan
    
display(diab_clean['Insulin'].isnull().mean())

0.4869791666666667

In [107]:
diab_clean = diab_clean.dropna(how='any', thresh=diab_clean.shape[0] * 0.7, axis=1)
display(diab_clean.head())

,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,98.0,58.0,33.0,34.0,0.430,43,0
1,2,112.0,75.0,32.0,35.7,0.148,21,0
2,2,108.0,64.0,NaN,30.8,0.158,21,0
3,8,107.0,80.0,NaN,24.6,0.856,34,0
4,7,136.0,90.0,NaN,29.9,0.210,50,0


In [108]:
thresh = diab_clean.shape[1]-2
diab_clean = diab_clean.dropna(how='any', thresh=thresh, axis=0)
diab_clean.shape


(761, 8)

In [112]:
values = {
    'Glucose': diab_clean['Glucose'].median(),
    'BloodPressure': diab_clean['BloodPressure'].median(),
    'SkinThickness': diab_clean['SkinThickness'].median(),
    'BMI': diab_clean['BMI'].median()
}
diab_clean = diab_clean.fillna(values)
diab_clean['SkinThickness'].mean()

29.109067017082786

In [127]:
def outliers_iqr_mod(data, feature, left=1.5, right=1.5, log_scale=False):
    """
    Давайте немного модифицируем нашу функцию outliers_iqr(). 
    Добавьте в нее параметры left и right, которые задают число IQR влево и вправо от границ ящика (пусть по умолчанию они равны 1.5).
    Функция, как и раньше должна возвращать потенциальные выбросы и очищенный DataFrame.
    """
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned

a, b = outliers_iqr_mod(diab_clean, 'SkinThickness')
display(a.shape)

(96, 8)

In [129]:
def outliers_z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned

c, d = outliers_z_score(diab_clean, 'SkinThickness')
c.shape

(4, 8)

In [133]:
e, f = outliers_iqr_mod(diab_clean, 'DiabetesPedigreeFunction')
g, h = outliers_iqr_mod(diab_clean, 'DiabetesPedigreeFunction', log_scale=True)
display(e.shape[0]-g.shape[0])


29